<a href="https://colab.research.google.com/github/desireechen/Deep-Learning/blob/master/Image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Alternative 1. Get the entire dset. No separation of training and validation. Everything is used for training.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/a-or-b.zip

In [ ]:
# Alternative 1A. Get the training and validation dset separately.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/a-or-b.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-a-or-b.zip

In [ ]:
# Alternative 2. Get the entire dset. Then, separate training and validation.
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

In [ ]:
# Alternative 2A. Get the entire dset which already has the data separated into training and validation.
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

In [ ]:
# Alternative 3. Multi-class classification.
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /tmp/rps.zip
  
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /tmp/rps-test-set.zip

In [ ]:
import tensorflow as tf
import os
import zipfile

In [ ]:
# Transfer Learning.
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 # Snapshot of the model after being trained.

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5' 
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), # This is the desired input shape of the data.
                                include_top = False, # Ignore the fully-connected layer at the top of InceptionV3.
                                weights = None) # I do not wish to use the built-in weights. I wish to use the snapshot that was downloaded. 
pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable = False # Lock the pre-trained layers of the model.
# pre_trained_model.summary()
last_layer = pre_trained_model.get_layer('mixed7')
print('Last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Alternative 1.
local_zip = '/tmp/a-or-b.zip' # Insert name of zip file.
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/a-or-b') 
zip_ref.close()

# Define each of the directories.
train_a_dir = os.path.join('/tmp/a-or-b/a') # Fill in the correct path. If the file is horses, then replace a with horses.
train_b_dir = os.path.join('/tmp/a-or-b/b') # Fill in the correct path. If the file is humans, then replace b with humans.

In [ ]:
# Alternative 1A and 3. For Alternative 1 above, only need half the lines. Remember to close the last line.
local_zip = '/tmp/a-or-b.zip' # Insert name of zip file.
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/a-or-b') ## For Alternative 3, just extract to ('/tmp/')
zip_ref.close()
local_zip = '/tmp/validation-a-or-b.zip' # Insert name of zip file.
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-a-or-b') ## For Alternative 3, just extract to ('/tmp/')
zip_ref.close()

# Define each of the directories.
train_a_dir = os.path.join('/tmp/a-or-b/a') # Fill in the correct path. If the file is horses, then replace a with horses.
train_b_dir = os.path.join('/tmp/a-or-b/b') # Fill in the correct path. If the file is humans, then replace b with humans.
rock_dir = os.path.join('/tmp/rps/rock') ## This is for Alternative 3. Do the same for paper and scissors.
validation_a_dir = os.path.join('/tmp/validation-a-or-b/a') # Fill in the correct path. If the file is horses, then replace a with horses.
validation_b_dir = os.path.join('/tmp/validation-a-or-b/b') # Fill in the correct path. If the file is humans, then replace b with humans.

In [ ]:
# Alternative 2. Separate training and validation myself.
local_zip = '/tmp/cats-and-dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Make each of the directories including base directory.
try:
  os.mkdir('/tmp/base_dir')
  os.mkdir('/tmp/base_dir/train_dir')
  os.mkdir('/tmp/base_dir/validation_dir')
  os.mkdir('/tmp/base_dir/train_dir/a/')
  os.mkdir('/tmp/base_dir/validation_dir/a/')
  os.mkdir('/tmp/base_dir/train_dir/b/')
  os.mkdir('/tmp/base_dir/validation_dir/b/')
except OSError:
  pass

import random
from shutil import copyfile
# Function to randomise files, then split into train and validation based on a split proportion.
def split_data(SOURCE, TRAIN, VALIDATION, SPLIT_SIZE):
  files = []
  for fname in os.listdir(SOURCE):
    path = SOURCE + fname # I am making the path, so that later on I can get the size of the file that is in the path.
    if os.path.getsize(path) > 0: # Checking if file is of zero length.
      files.append(fname)
    else:
      print(fname + " is zero length, so ignoring.")

  train_length = int(len(files) * SPLIT_SIZE)
  validation_length = int(len(files) - train_length)
  shuffled_set = random.sample(files, len(files))
  train_set = shuffled_set[0:train_length]
  validation_set = shuffled_set[-validation_length:]

  for fname in train_set:
    source = SOURCE + fname
    destination = TRAIN + fname
    copyfile(source, destination)

  for fname in validation_set:
    source = SOURCE + fname
    destination = VALIDATION + fname
    copyfile(source, destination)

# Define arguments in the function.
a_source_dir = '/tmp/PetImages/a/' # Insert path accordingly. If the file is Cat, then replace a with Cat.
train_a_dir = '/tmp/base_dir/train_dir/a/'
validation_a_dir = '/tmp/base_dir/validation_dir/a/'
b_source_dir = '/tmp/PetImages/b/' # Insert path accordingly. If the file is Dog, then replace b with Dog.
train_b_dir = '/tmp/base_dir/train_dir/b/'
validation_b_dir = '/tmp/base_dir/validation_dir/b/'
split_size = 0.9

split_data(a_source_dir, train_a_dir, validation_a_dir, split_size)
split_data(b_source_dir, train_b_dir, validation_b_dir, split_size)

In [ ]:
# Alternative 2A. Entire dset is separated into training and validation.
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Define each of the directories.
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
train_a_dir = os.path.join(train_dir, 'a') # Insert path accordingly. If the file is Cat, then replace a with Cat.
train_b_dir = os.path.join(train_dir, 'b') # Insert path accordingly. If the file is Dog, then replace b with Dog.
validation_a_dir = os.path.join(validation_dir, 'a') # Insert path accordingly. If the file is Cat, then replace a with Cat.
validation_b_dir = os.path.join(validation_dir, 'b') # Insert path accordingly. If the file is Dog, then replace b with Dog.

In [ ]:
# Alternative 4. Using csv files.
import csv
import numpy as np

uploaded = files.upload()

def get_data(filename):
    with open(filename) as training_file:
        csv_reader = csv.reader(training_file, delimiter=',')
        first_line = True
        temp_images = []
        temp_labels = []
        for row in csv_reader:
            if first_line:
              first_line = False # Ignore the 1st line containing the column headers.
            else:
                temp_labels.append(row[0]) # This has the label of the image.
                image_data = row[1:785] # These are the pixel values for the images.
                image_data_as_array = np.array_split(image_data, 28) # Turn the 784 pixels into 28x28.
                temp_images.append(image_data_as_array)
        images = np.array(temp_images).astype('float') # Data is in strings. Need to convert to float.
        labels = np.array(temp_labels).astype('float') # Data is in strings. Need to convert to float.
    return images, labels

training_images, training_labels = get_data('sign_mnist_train.csv')
testing_images, testing_labels = get_data('sign_mnist_test.csv')

# print(training_images.shape)
# print(training_labels.shape)
# print(testing_images.shape)
# print(testing_labels.shape)

In [ ]:
# Extra code. 
# See the filenames using os.listdir
train_a_fnames = os.listdir(train_a_dir)
print(train_a_fnames[:10])
train_b_names = os.listdir(train_b_dir)
print(train_b_names[:10])
validation_a_fnames = os.listdir(validation_a_dir)
print(validation_a_fnames[:10])
validation_b_names = os.listdir(validation_b_dir)
print(validation_b_names[:10])

rock_files = os.listdir(rock_dir) ## This is for Alternative 3. 
print(rock_files[:10])
paper_files = os.listdir(paper_dir) ## This is for Alternative 3. 
print(rock_files[:10])
scissors_files = os.listdir(scissors_dir) ## This is for Alternative 3. 
print(rock_files[:10])

# See the total number of images in each of the directories.
print('total training A images:', len(os.listdir(train_a_dir)))
print('total training B images:', len(os.listdir(train_b_dir)))
print('total validation A images:', len(os.listdir(validation_a_dir)))
print('total validation B images:', len(os.listdir(validation_b_dir)))

# See the total number of images in each of the paths.
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))
print(len(os.listdir('/tmp/base_dir/train_dir/a/')))
print(len(os.listdir('/tmp/base_dir/train_dir/b/')))
print(len(os.listdir('/tmp/base_dir/validation_dir/a/')))
print(len(os.listdir('/tmp/base_dir/validation_dir/b/')))

# See the size of the file. Check if the file is of zero length.
os.path.getsize('/tmp/base_dir/train_dir/train_a_dir/2354.jpg')

In [ ]:
# Extra code. Take a look at the images. This is for binary classification.
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Set graph to output images in 4-by-4 configuration.
nrows = 4
ncols = 4
pic_index = 0 # Index for iterating over images.
# Set up matplotlib fig and size it to fit 4-by-4 configuration.
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index += 8
next_a_pix = [os.path.join(train_a_dir, fname) for fname in train_a_fnames[pic_index-8:pic_index]]
next_b_pix = [os.path.join(train_b_dir, fname) for fname in train_b_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_a_pix+next_b_pix):
  # Set up subplot.
  sp = plt.subplot(nrows, ncols, i+1)
  sp.axis('Off') # I do not wish to show gridlines.
  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
# Extra code. Take a look at the images. This is for MULTI-CLASS CLASSIFICATION.
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2
next_rock = [os.path.join(rock_dir, fname) for fname in rock_files[pic_index-2:pic_index]]
next_paper = [os.path.join(paper_dir, fname) for fname in paper_files[pic_index-2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) for fname in scissors_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_rock+next_paper+next_scissors):
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

In [ ]:
# Data Preprocessing. Alternatives below.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Alternative AUGMENTATION. Validation data should not be augmented.
train_datagen = ImageDataGenerator(
    rescale = 1./255, # Another has it as rescale = 1./255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest' # Some do not have this line.
)

In [ ]:
# Alternative 1. Data Preprocessing. Set up data generators that will read images from subdirectories and auto label with the subdirectory name. 
train_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/a-or-b/', # Point the generator to the directory that contains the sub-directories.
    target_size=(300,300), # This is ensuring that images are uniformly resized. Images are resized as they are loaded. No change to the source image.
                            # If compact, then use (150,150) or whatever.
    batch_size=128,
    class_mode='binary' # We need binary labels for binary classification problem. 
)

In [ ]:
# Alternative 1A. Data Preprocessing. Set up data generators that will read images from subdirectories and auto label with the subdirectory name.  
train_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/tmp/a-or-b/', # Point the generator to the directory that contains the sub-directories.
    target_size=(300,300), # This is ensuring that images are uniformly resized. Images are resized as they are loaded. No change to the source image.
                            # If compact, then use (150,150) or whatever.
    batch_size=128,
    class_mode='binary' # We need binary labels for binary classification problem. 
)

validation_generator = validation_datagen.flow_from_directory(
    '/tmp/validation-a-or-b/', # Point the generator to the directory that contains the sub-directories.
    target_size=(300,300), # This is ensuring that images are uniformly resized. Images are resized as they are loaded. No change to the source image.
                            # If compact, then use (150,150) or whatever.
    batch_size=32,
    class_mode='binary' # We need binary labels for binary classification problem. 
)

In [ ]:
# Alternative 2 and 2A. Data Preprocessing. Set up data generators that will read images from subdirectories and auto label with the subdirectory name.  
train_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)

train_dir = '/tmp/base_dir/train_dir/' ## This line is not required for Alternative 2A.
train_generator = train_datagen.flow_from_directory(
    train_dir, # Point the generator to the directory called Train that contains the sub-directories A or B.
    target_size=(300,300), # This is ensuring that images are uniformly resized. Images are resized as they are loaded. No change to the source image.
                            # If compact, then use (150,150) or whatever.
    batch_size=20, # If there are 1000 images and batch size is 20, there will be 50 batches.
    class_mode='binary' # We need binary labels for binary classification problem. 
)

validation_dir = '/tmp/base_dir/validation_dir/' ## This line is not required for Alternative 2A.
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, # Point the generator to the directory called Train that contains the sub-directories A or B.
    target_size=(300,300), # This is ensuring that images are uniformly resized. Images are resized as they are loaded. No change to the source image.
                            # If compact, then use (150,150) or whatever.
    batch_size=20, # If there are 1000 images and batch size is 20, there will be 50 batches.
    class_mode='binary' # We need binary labels for binary classification problem. 
)

In [ ]:
# Alternative 3. Data Preprocessing. Set up data generators that will read images from subdirectories and auto label with the subdirectory name. 
train_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)

train_dir = '/tmp/rps/'
train_generator = train_datagen.flow_from_directory(
    train_dir, # Point the generator to the directory called Train that contains the sub-directories A or B.
    target_size=(300,300), # This is ensuring that images are uniformly resized. Images are resized as they are loaded. No change to the source image.
                            # If compact, then use (150,150) or whatever.
    batch_size=20, # If there are 1000 images and batch size is 20, there will be 50 batches.
    class_mode='categorical' ## This is different from binary classification.
)

validation_dir = '/tmp/rps-test-set/'
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, # Point the generator to the directory called Train that contains the sub-directories A or B.
    target_size=(300,300), # This is ensuring that images are uniformly resized. Images are resized as they are loaded. No change to the source image.
                            # If compact, then use (150,150) or whatever.
    batch_size=20, # If there are 1000 images and batch size is 20, there will be 50 batches.
    class_mode='categorical' ## This is different from binary classification.
)

In [ ]:
# Alternative 4. Data Preprocessing. Set up data generators that will read images from subdirectories and auto label with the subdirectory name.
# Need to add a dimension to the data. Make (10000, 28, 28) into (10000, 28, 28, 1)
training_images = np.expand_dims(training_images, axis=3)
testing_images = np.expand_dims(testing_images, axis=3)

train_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1/255) # Another has it as (rescale=1.0/255. ) or (rescale=1./255)

train_generator = train_datagen.flow(training_images, training_labels, batch_size=32) ## This line is subsumed later.
validation_generator = validation_datagen.flow(testing_images, testing_labels, batch_size=32) ## This line is subsumed later.

# print(training_images.shape)
# print(testing_images.shape)

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(300,300,3)),
                                    # 1st argument above is no. of convolutions. This number can be 16, 32, 64 or 128.
                                    # relu means returning x when x>0, else return 0.
                                    # The Conv2D filter is 3-by-3. Output is 2 pixels smaller on x and y.
                                    # If the filter is 5-by-5, output is 4 pixels smaller on x and y.
                                    # input_shape is the desired size of the image 300x300 in colour. 
                                    ## I can compact input_shape to (150,150,3) or other values.
                                    ## For Alternative 4, input_shape=(28,28,1) for sign language dataset.
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(16, (3,3), activation='relu'), # I can add more convolutions.
                                    tf.keras.layers.MaxPooling2D(2,2), # I can add more pooling layers.
                                    tf.keras.layers.Conv2D(32, (3,3), activation='relu'), # I can add more convolutions.
                                    tf.keras.layers.MaxPooling2D(2,2), # I can add more pooling layers.
                                    tf.keras.layers.Dropout(0.5), # This is the OPTIONAL dropout layer. This number can be 0.2 or 0.5 etc.
                                    # Dropouts remove a random number of neurons in your neural network. Dropout is a regularization technique.
                                    tf.keras.layers.Flatten(), # Flatten to feed into the Dense Neural Network.
                                    # tf.keras.layers.Dropout(0.5), # This is the OPTIONAL dropout layer. This number can be 0.2 or 0.5 etc.
                                    # Dropouts remove a random number of neurons in your neural network. Dropout is a regularization technique.
                                    tf.keras.layers.Dense(128, activation='relu'), # This number can be 64, 128, 256, 512 or 1024.
                                    # tf.keras.layers.Dropout(0.5), # This is the OPTIONAL dropout layer. This number can be 0.2 or 0.5 etc.
                                    # Dropouts remove a random number of neurons in your neural network. Dropout is a regularization technique.
                                    tf.keras.layers.Dense(1, activation='sigmoid')]) # Only 1 output neuron for binary classification problem.
                                    # Output is a single scalar between 0 and 1 for binary classification problem.
                                    # tf.keras.layers.Dense(26, activation='softmax') 
                                    ## This is for Alternatives 3 and 4 which has 3 and 26 categories respectively.
# model.summary()

In [ ]:
# Alternatives 1, 1A, 2 and 2A.
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001), #RMSprop automatically adapts the learning rate tuning for me. This number can be 0.001 or 1e-4.
              loss='binary_crossentropy', # Use this for binary classification problem.
              metrics=['accuracy'])

In [ ]:
# Alternative 3.
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001), #RMSprop automatically adapts the learning rate tuning for me. This number can be 0.001 or 1e-4.
              # optimizer='rmsprop',
              loss='categorical_crossentropy', # Use this for multi-class classification problem.
              metrics=['accuracy'])

In [ ]:
# Alternative 4.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) # optimizer=tf.optimizers.Adam()

In [ ]:
# Build model when using transfer learning.
x = layers.Flatten()(last_output)
x = layers.Dense(128, activation='relu')(x) # This number can be 64, 128, 256, 512 or 1024.
x = layers.Dropout(0.2)(x) # Dropping out 20% of the neurons. This number can be 0.2 or 0.5 etc.
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001), #RMSprop automatically adapts the learning rate tuning for me. This number can be 0.001 or 1e-4.
              loss='binary_crossentropy', # Use this for binary classification problem.
              metrics=['accuracy'])
# model.summary() # Output expected to be very large.

In [ ]:
# Alternative CALLBACKS based on accuracy.
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.6): # This number can be 0.99, 0.998 or 0.999. Can also use val_accuracy.
      print("\nReached 60% accuracy so cancelling training!") # Need to change the print accordingly.
      self.model.stop_training = True

In [ ]:
# Alternative CALLBACKS based on loss.
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss')<0.4): # Can also use val_loss.
      print("\nReached less than 40% loss so cancelling training!") # Need to change the print accordingly.
      self.model.stop_training = True

In [ ]:
# Alternative 1. Without callbacks.
history = model.fit(train_generator, steps_per_epoch=8, epochs=15, verbose=1) 
# Training batch size previously was 128 multiplied by steps_per_epoch to go through all the training images.
# verbose 0, 1, 2 shows the least, most, in between information.
# verbose 1 or 2 will show training loss, training accuracy, validation loss and validation accuracy. 

In [ ]:
# Alternatives 1A, 2, 2A and 3. Without callbacks.
history = model.fit(train_generator, steps_per_epoch=8, epochs=15, verbose=1, validation_data=validation_generator, validation_steps=8)
# Training batch size previously was 128 multiplied by steps_per_epoch to go through all the training images.
# Validation batch size previously was 32 multiplied by validation_steps to go through all the validation images.
# verbose 0, 1, 2 shows the least, most, in between information.
# verbose 1 or 2 will show training loss, training accuracy, validation loss and validation accuracy. 

In [ ]:
# Alternative 4. Without callbacks.
history = model.fit(train_generator, steps_per_epoch=len(training_images)/32, epochs=15, validation_data=validation_generator, validation_steps=len(testing_images)/32)
model.evaluate(testing_images, testing_labels)

In [ ]:
# Alternative 1 with callbacks.
callbacks = myCallback()
history = model.fit(train_generator, steps_per_epoch=8, epochs=15, verbose=1, callbacks=[callbacks]) 
# Training batch size previously was 128 multiplied by steps_per_epoch to go through all the training images.
# verbose 0, 1, 2 shows the least, most, in between information.
# verbose 1 or 2 will show training loss, training accuracy, validation loss and validation accuracy. 

In [ ]:
# Alternatives 1A, 2, 2A and 3 with callbacks.
callbacks = myCallback()
history = model.fit(train_generator, steps_per_epoch=8, epochs=15, verbose=1, validation_data=validation_generator, validation_steps=8, callbacks=[callbacks])
# Training batch size previously was 128 multiplied by steps_per_epoch to go through all the training images.
# Validation batch size previously was 32 multiplied by validation_steps to go through all the validation images.
# verbose 0, 1, 2 shows the least, most, in between information.
# verbose 1 or 2 will show training loss, training accuracy, validation loss and validation accuracy. 

In [ ]:
# Alternative 4 with callbacks.
callbacks = myCallback()
history = model.fit(train_generator, steps_per_epoch=len(training_images)/32, epochs=15, validation_data=validation_generator, validation_steps=len(testing_images)/32, callbacks=[callbacks])
model.evaluate(testing_images, testing_labels)

In [ ]:
# Extra code. To save model.
model.save("rps.h5")

In [ ]:
# Extra code. To upload my own files for the model to predict. I have not tidied up much of this code.
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300,300)) # The target size needs to match the input size of the model. 
                                                    # If compact, then use (150,150) or whatever.
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human") # Change the printing accordingly.
  else:
    print(fn + " is a horse") # Change the printing accordingly.

In [ ]:
# Extra code. To visualise intermediate representations. I have not tidied up much of this code.
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output 
# intermediate representations for all layers in the previous model after the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
# visualization_model = Model(img_input, successive_outputs)
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)
# Let's prepare a random input image from the training set.
a_img_files = [os.path.join(train_a_dir, fname) for fname in train_a_fnames]
b_img_files = [os.path.join(train_b_dir, fname) for fname in train_b_fnames]
img_path = random.choice(a_img_files + b_img_files)
img = load_img(img_path, target_size=(300, 300)) # If compact, then use (150,150) or whatever. The shape below will change accordingly.
x = img_to_array(img)  # Numpy array with shape (300, 300, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 300, 300, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc)) # Get the number of epochs.

plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation accuracy')
plt.legend(loc=0) # This is to place legend at bottom right of graph.
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation loss')
plt.legend() 

plt.show()